In [1]:
import torch
import torch.nn as nn
from mtl_pytorch import layer_node, mtl_model, mobilenetv2
import numpy as np
from framework.trainer import Trainer
from torch import optim

import numpy as np
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from data.heads.pixel2pixel import ASPPHeadNode

from data.dataloader.cityscapes_dataloader import CityScapes
from data.metrics.pixel2pixel_loss import CityScapesCriterions
from data.metrics.pixel2pixel_metrics import CityScapesMetrics

# define a model

In [9]:
# build model
class MTL_network(nn.Module):
    def __init__(self):
        super(MTL_network, self).__init__()
        self.convNode = layer_node.Conv2dNode(
            in_channels=1, out_channels=128, kernel_size=3,
            taskList=['segment_sementic'])

    def forward(self, x, stage='common', task=None, tau=5, hard=False):
        x = self.convNode(x, stage, task, tau, hard)
        x = x.view(-1, 256)
        x = nn.Linear(256, 1)(x)
        return x

# Basic output

In [17]:
from copy import deepcopy
# give a basic input and output
model = MTL_network()
inp = np.array([[[[1, 2, 3, 4],
                  [1, 2, 3, 4],
                  [1, 2, 3, 4]]]])
print(inp.shape)
x = torch.tensor(inp, dtype=torch.float)
print(model(x))
for name, module in deepcopy(model).named_modules():
    model.__setattr__(name, nn.Module())

for module in model.modules():
    print(module)

(1, 1, 3, 4)
tensor([[-0.0284]], grad_fn=<AddmmBackward0>)
MTL_network(
  (convNode): Module()
  (): Module()
  (convNode.taskOp): Module()
  (convNode.basicOp): Module()
  (convNode.policy): Module()
  (convNode.dsOp): Module()
)
Module()
Module()
Module()
Module()
Module()
Module()


# Prepare for training

In [4]:
tasks = ['segment_semantic', 'normla', 'depth_zbuffer']

# Train

In [5]:
checkpoint = 'checkpoint/'
trainer = Trainer(
    model, train_dataloader_dict={}, val_dataloader_dict={},
    criterion_dict={}, metric_dict={}, print_iters=10,
    val_iters=10,save_num=1, policy_update_iters=10)

## pre_train

In [6]:
trainer.pre_train(iters=10, lr=0.0001,
           savePath=checkpoint+'mtl_pytorch/')

ZeroDivisionError: division by zero

## alter_train